In [5]:
import requests
import pandas as pd
import json
from datetime import datetime, timedelta
import time
import os

class PolymarketPriceHistory:
    def __init__(self, base_url="https://clob.polymarket.com"):
        self.base_url = base_url
        self.headers = {
            'Content-Type': 'application/json',
            'User-Agent': 'Mozilla/5.0 (compatible; PolymarketAPI/1.0)'
        }
    
    def get_market_info(self, event_slug):
        """
        Get market information for an event to find token IDs.
        
        Args:
            event_slug (str): Event slug like "nfl-lac-hou-2025-01-11"
        
        Returns:
            dict: Market information with token IDs
        """
        
        # First, get markets for this event
        markets_url = f"{self.base_url}/markets"
        
        try:
            response = requests.get(markets_url, headers=self.headers)
            response.raise_for_status()
            markets = response.json()
            
            # Find market matching our event slug
            target_market = None
            for market in markets:
                if event_slug in market.get('question', '').lower() or \
                   event_slug in str(market.get('id', '')):
                    target_market = market
                    break
            
            if target_market:
                print(f"Found market: {target_market.get('question', 'Unknown')}")
                print(f"Market ID: {target_market.get('id')}")
                print(f"Tokens: {target_market.get('tokens', [])}")
                return target_market
            else:
                print(f"Market not found for event: {event_slug}")
                return None
                
        except Exception as e:
            print(f"Error fetching market info: {e}")
            return None
    
    def get_price_history(self, token_id, start_ts=None, end_ts=None, interval=None, fidelity=1):
        """
        Get price history for a specific token.
        
        Args:
            token_id (str): CLOB token ID
            start_ts (int): Start timestamp (Unix, UTC)
            end_ts (int): End timestamp (Unix, UTC)
            interval (str): Time interval ("1m", "1w", "1d", "6h", "1h", "max")
            fidelity (int): Resolution in minutes (1 = 1 minute intervals)
        
        Returns:
            pd.DataFrame: Price history data
        """
        
        url = f"{self.base_url}/prices-history"
        
        params = {
            'market': token_id,
            'fidelity': fidelity
        }
        
        # Use either timestamp range OR interval (mutually exclusive)
        #if start_ts and end_ts:
        #    params['startTs'] = start_ts
        #    params['endTs'] = end_ts
        #elif interval:
        #    params['interval'] = interval
        #else:
            # Default to last 7 days
        params['interval'] = 'all'
        
        try:
            print(f"Fetching price history for token {token_id}...")
            print(f"Parameters: {params}")
            
            response = requests.get(url, params=params, headers=self.headers)
            response.raise_for_status()
            
            data = response.json()

            print(data)
            
            if 'history' in data and data['history']:
                # Convert to DataFrame
                df = pd.DataFrame(data['history'])
                
                # Convert timestamp to datetime
                df['datetime'] = pd.to_datetime(df['t'], unit='s')
                df['price'] = df['p']
                
                # Sort by time
                df = df.sort_values('datetime').reset_index(drop=True)
                
                print(f"Retrieved {len(df)} price points")
                print(f"Date range: {df['datetime'].min()} to {df['datetime'].max()}")
                
                return df[['datetime', 'price', 't', 'p']]
            
            else:
                print("No price history data found")
                return pd.DataFrame()
                
        except Exception as e:
            print(f"Error fetching price history: {e}")
            return pd.DataFrame()
    
    def get_nfl_market_prices(self, event_slug="nfl-lac-hou-2025-01-11", days_back=7):
        """
        Get price data for the NFL LAC vs HOU market.
        
        Args:
            event_slug (str): Event identifier
            days_back (int): Number of days back to fetch data
        
        Returns:
            dict: Price data for both outcomes
        """
        
        # For the specific NFL market, we need to find the token IDs
        # This is a simplified approach - you may need to adjust based on actual market structure
        
        # Calculate time range
        end_time = datetime.now()
        start_time = end_time - timedelta(days=days_back)
        
        start_ts = int(start_time.timestamp())
        end_ts = int(end_time.timestamp())
        
        print(f"Fetching data from {start_time} to {end_time}")
        
        # These are example token IDs - you'll need to find the actual ones
        # You can get them from the markets endpoint or by inspecting the web interface
        example_token_ids = [
            "21742633143463906290569050155826241533067272736897614950488156847949938836455",  # Example: LAC wins
            "21742633143463906290569050155826241533067272736897614950488156847949938836456"   # Example: HOU wins
        ]
        
        results = {}
        
        # Try to get actual market info first
        market_info = self.get_market_info(event_slug)
        if market_info and 'tokens' in market_info:
            token_ids = [token['token_id'] for token in market_info['tokens']]
            token_names = [token.get('outcome', f"Outcome {i}") for i, token in enumerate(market_info['tokens'])]
        else:
            # Fallback to example IDs
            print("Using example token IDs - you may need to update these")
            token_ids = example_token_ids
            token_names = ["LAC wins", "HOU wins"]
        
        for i, token_id in enumerate(token_ids):
            outcome_name = token_names[i] if i < len(token_names) else f"Outcome {i}"
            
            print(f"\nFetching prices for {outcome_name} (Token: {token_id})")
            
            df = self.get_price_history(
                token_id=token_id,
                start_ts=start_ts,
                end_ts=end_ts,
                fidelity=1  # 1-minute intervals
            )
            
            if not df.empty:
                results[outcome_name] = df
            
            # Be respectful with API calls
            time.sleep(0.5)
        
        return results
    
    def save_price_data(self, price_data, filename_prefix="nfl_lac_hou"):
        """
        Save price data to CSV files.
        
        Args:
            price_data (dict): Dictionary of DataFrames from get_nfl_market_prices
            filename_prefix (str): Prefix for saved files
        """
        
        data_dir = "data"
        os.makedirs(data_dir, exist_ok=True)
        
        timestamp = datetime.now().strftime("%Y%m%d_%H%M")
        
        saved_files = []
        
        for outcome_name, df in price_data.items():
            # Clean outcome name for filename
            clean_name = outcome_name.lower().replace(" ", "_").replace("/", "_")
            filename = f"{filename_prefix}_{clean_name}_{timestamp}.csv"
            filepath = os.path.join(data_dir, filename)
            
            df.to_csv(filepath, index=False)
            saved_files.append(filepath)
            
            print(f"Saved {len(df)} price points for '{outcome_name}' to {filepath}")
        
        return saved_files
    
    def plot_price_data(self, price_data):
        """
        Create a simple plot of the price data.
        
        Args:
            price_data (dict): Dictionary of DataFrames
        """
        try:
            import matplotlib.pyplot as plt
            
            plt.figure(figsize=(12, 8))
            
            for outcome_name, df in price_data.items():
                if not df.empty:
                    plt.plot(df['datetime'], df['price'], label=outcome_name, linewidth=2)
            
            plt.title('NFL LAC vs HOU - Price History (1-minute intervals)')
            plt.xlabel('Time')
            plt.ylabel('Price')
            plt.legend()
            plt.grid(True, alpha=0.3)
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.show()
            
        except ImportError:
            print("matplotlib not available for plotting")

# Example usage
def main():
    """
    Main function to demonstrate fetching NFL market price data.
    """
    
    # Initialize the API client
    api = PolymarketPriceHistory()
    
    # Fetch price data for the NFL LAC vs HOU market
    print("Fetching NFL LAC vs HOU price data...")
    
    price_data = api.get_nfl_market_prices(
        event_slug="nfl-lac-hou-2025-01-11",
        days_back=7  # Last 7 days of data
    )
    
    if price_data:
        print(f"\nSuccessfully fetched data for {len(price_data)} outcomes")
        
        # Save to CSV files
        saved_files = api.save_price_data(price_data)
        
        # Display summary
        for outcome_name, df in price_data.items():
            if not df.empty:
                print(f"\n{outcome_name}:")
                print(f"  Data points: {len(df)}")
                print(f"  Price range: ${df['price'].min():.3f} - ${df['price'].max():.3f}")
                print(f"  Latest price: ${df['price'].iloc[-1]:.3f}")
                print(f"  First 5 records:")
                print(df[['datetime', 'price']].head())
        
        # Optional: Create plots
        api.plot_price_data(price_data)
        
    else:
        print("No price data retrieved")

if __name__ == "__main__":
    main()

# Additional utility functions
def fetch_custom_date_range(token_id, start_date, end_date):
    """
    Fetch price data for a custom date range.
    
    Args:
        token_id (str): CLOB token ID
        start_date (str): Start date in YYYY-MM-DD format
        end_date (str): End date in YYYY-MM-DD format
    
    Returns:
        pd.DataFrame: Price history
    """
    
    api = PolymarketPriceHistory()
    
    start_ts = int(datetime.strptime(start_date, "%Y-%m-%d").timestamp())
    end_ts = int(datetime.strptime(end_date, "%Y-%m-%d").timestamp())
    
    return api.get_price_history(
        token_id=token_id,
        start_ts=start_ts,
        end_ts=end_ts,
        fidelity=1
    )

# Example of fetching specific date range:
df = fetch_custom_date_range(
    token_id="27437865304614543548776035900279780911310936329907359892446239360881301538755",
    start_date="2025-04-15",
    end_date="2025-05-15"
)
27437865304614543548776035900279780911310936329907359892446239360881301538755

Fetching NFL LAC vs HOU price data...
Fetching data from 2025-05-24 13:39:13.967377 to 2025-05-31 13:39:13.967377
Error fetching market info: 'str' object has no attribute 'get'
Using example token IDs - you may need to update these

Fetching prices for LAC wins (Token: 21742633143463906290569050155826241533067272736897614950488156847949938836455)
Fetching price history for token 21742633143463906290569050155826241533067272736897614950488156847949938836455...
Parameters: {'market': '21742633143463906290569050155826241533067272736897614950488156847949938836455', 'fidelity': 1, 'interval': 'all'}
{'history': []}
No price history data found

Fetching prices for HOU wins (Token: 21742633143463906290569050155826241533067272736897614950488156847949938836456)
Fetching price history for token 21742633143463906290569050155826241533067272736897614950488156847949938836456...
Parameters: {'market': '21742633143463906290569050155826241533067272736897614950488156847949938836456', 'fidelity': 1, 'int